In [1]:
import joblib

label_encoders = joblib.load('../models/label_encoders.pkl')
scaler = joblib.load('../models/scaler.pkl')
model = joblib.load('../models/random_forest_model.pkl')

In [2]:
# Function to preprocess new data
def preprocess_new_data(data, label_encoders, scaler):
    # Encode categorical variables
    for column, le in label_encoders.items():
        data[column] = le.transform(data[column])
    
    # Normalize numerical features
    data_scaled = scaler.transform(data)
    
    return data_scaled

In [3]:
import pandas as pd
df = pd.read_csv('data_live_engineer_filtered.csv', encoding='utf-8')

In [4]:
Xht = df.tail(1).drop(columns=['result'])

In [12]:
df.tail(1)['result']

26883    0
Name: result, dtype: int64

In [5]:
# Preprocess the new data
new_data_preprocessed = preprocess_new_data(Xht, label_encoders, scaler)

# Make predictions
predictions = model.predict(new_data_preprocessed)

c:\Python310\lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
c:\Python310\lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
c:\Python310\lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
c:\Python310\lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
c:\Python310\lib\site-packages\s

In [7]:
predictions[0]

0

In [15]:
# Implementar o agente de Q-learning
class QLearningAgent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.q_table = np.zeros((state_size, action_size))
        self.gamma = 0.95
        self.epsilon = 1.0
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.995
        self.learning_rate = 0.01
        self.memory = []

    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return np.random.choice(self.action_size)
        return np.argmax(self.q_table[state])

    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def replay(self, batch_size):
        if len(self.memory) < batch_size:
            return
        minibatch = np.random.choice(len(self.memory), batch_size, replace=False)
        for index in minibatch:
            state, action, reward, next_state, done = self.memory[index]
            target = reward if done else reward + self.gamma * np.max(self.q_table[next_state])
            self.q_table[state][action] += self.learning_rate * (target - self.q_table[state][action])
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

import pickle

# Carregar o agente salvo
with open('../models/best_q_learning_agent.pkl', 'rb') as f:
    saved_agent = pickle.load(f)

# Função para simular apostas ao vivo com o agente salvo
def simulate_with_saved_agent(agent, X_new):
    betting_stats = []
    for i in range(len(X_new)):
        state = prepare_state(X_new[i])
        action = agent.act(state)
        betting_stats.append((X_new[i], action))
    return betting_stats

# Função para preparar estado
def prepare_state(data_row):
    return int(data_row.sum()) % state_size


In [17]:
# Função para preparar dados
def prepare_data(df):
    X = df.drop(columns=['result', 'homeTeam', 'awayTeam', 'match_id'])
    y = df['result']
    return X, y

# Função para criar e aplicar o transformador de colunas
def create_preprocessor(X):
    numeric_features = X.select_dtypes(exclude=['object']).columns.tolist()
    categorical_features = ['league']

    preprocessor = ColumnTransformer(
        transformers=[
            ('num', StandardScaler(), numeric_features),
            ('cat', OneHotEncoder(drop='first'), categorical_features)
        ])
    return preprocessor.fit(X)

# Função para simular apostas ao vivo com os novos parâmetros
def simulate_live_betting(agent, X_test, y_test):
    total_money = 0
    betting_stats = []
    for i in range(len(X_test)):
        state = prepare_state(X_test[i])
        action = agent.act(state)
        if action == 1:  # Fez uma aposta
            reward = 100 if y_test.iloc[i] == 1 else -100
            total_money += reward
        else:
            reward = 0
        next_state = prepare_state(X_test[(i + 1) % len(X_test)])
        agent.remember(state, action, reward, next_state, reward != 0)
        agent.replay(batch_size)
        betting_stats.append((X_test[i], action, reward))
    return total_money, betting_stats

# Função para preparar estado
def prepare_state(data_row):
    return int(data_row.sum()) % state_size

# Função para transformar de volta para a forma original
def inverse_transform(preprocessor, X_transformed):
    num_features = preprocessor.named_transformers_['num']
    cat_features = preprocessor.named_transformers_['cat']
    
    X_num_inv = num_features.inverse_transform(X_transformed[:, :num_features.n_features_in_])
    X_cat_inv = cat_features.inverse_transform(X_transformed[:, num_features.n_features_in_:])
    
    return np.hstack((X_num_inv, X_cat_inv))

In [78]:
import pandas as pd
df = pd.read_csv('data_live_engineer_filtered.csv', encoding='utf-8')

In [79]:
# ordenando as colunas
colunas = ['minute','shots_home', 'shots_away', 'league',
            'corners_home', 'corners_away', 'shotsOffgoal_home',
            'shotsOffgoal_away', 'fouls_home', 'fouls_away', 'tackles_home',
            'tackles_away', 'possessiontime_away',
            'possessiontime_home', 'f_attack_home', 'f_defensive_away',
            'f_defensive_home', 'f_attack_away', 'win_rate_home', 'loss_rate_home',
            'draw_rate_home', 'win_rate_away', 'loss_rate_away', 'draw_rate_away',
            'shotAccuracy_home', 'shotAccuracy_away', 'attackPressureOverTime_home',
            'attackPressureOverTime_away', 'aggrressionOverTime_home',
            'aggresssionOverTime_away', 'defensiveEfficacy_home', 'taklesOverTime_home', 'taklesOverTime_away',
            'possessionControl', 'passRisk_home', 'passRisk_away', '05ht_home',
            '05ht_away']

Xht = df.drop(columns=['result', 'homeTeam', 'awayTeam', 'match_id'])

In [61]:
l = int(Xht.shape[0])
l

26362

In [81]:
Xht.shape

(26362, 38)

In [90]:
import numpy as np
state_size = 20

X_new = preprocessor.transform(Xht.head(1))

# Exemplo de uso com novos dados
# X_new seria os novos dados de entrada transformados
betting_stats_new = simulate_with_saved_agent(saved_agent, X_new)

print(betting_stats_new[0][1])

1


c:\Python310\lib\site-packages\sklearn\utils\validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
c:\Python310\lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
c:\Python310\lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
c:\Python310\lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
c:\Python310\lib\site-pac

In [89]:
df.head(1)['result']

0    1
Name: result, dtype: int64

In [32]:
# Criar uma sessão
session = requests.Session()

In [40]:
import requests
# https://playscores.sportsat.app/gateway/api/v1/fixtures-svc/v2/fixtures/livescores?include=league,stats,pressureStats&take=3000
url = "https://playscores.sportsat.app/gateway/api/v1/fixtures-svc/v2/fixtures/livescores?include=league,stats,pressureStats&take=3000"

querystring = {"matchSlug": "\"31-01-2023-union-berlin-vfl-wolfsburg\"", "sportSlug": "soccer",
                "include": "weatherReport,additionalInfo,league,stats,pressureStats,probabilities"}

payload = ""
headers = {
    'Accept': 'application/json, text/plain, */*',
    'Accept-Encoding': 'gzip, deflate, br, zstd',
    'Accept-Language': 'pt-BR,pt;q=0.9,en;q=0.8',
    'If-Modified-Since': 'Sun, 28 Jul 2024 17:44:41 GMT',
    'Origin': 'https://www.playscores.com',
    'Referer': 'https://www.playscores.com/',
    'Sec-Ch-Ua': '"Not_A Brand";v="8", "Chromium";v="120", "Google Chrome";v="120"',
    'Sec-Ch-Ua-Mobile': '?0',
    'Sec-Ch-Ua-Platform': '"Windows"',
    'Sec-Fetch-Dest': 'empty',
    'Sec-Fetch-Mode': 'cors',
    'Sec-Fetch-Site': 'cross-site',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36'

}

response = session.request(
    "GET", url, data=payload, headers=headers)
response

<Response [200]>

In [43]:
dic_response = response.json()
numero_jogos = len(dic_response['data'])
print(f'🤖 {numero_jogos} jogos ao vivo\n')

🤖 44 jogos ao vivo



In [48]:
for game in dic_response['data']:
    print(f"{game['stats']}")

{'fixtureId': 19134671, 'attacks': {'home': 38, 'away': 59}, 'ballSafe': {'home': None, 'away': None}, 'corners': {'home': 1, 'away': 6}, 'dangerousAttacks': {'home': 11, 'away': 44}, 'fouls': {'home': None, 'away': None}, 'freeKick': {'home': None, 'away': None}, 'goalAttempts': {'home': None, 'away': None}, 'goalKick': {'home': None, 'away': None}, 'goals': {'home': 0, 'away': 1}, 'injuries': {'home': None, 'away': None}, 'offsides': {'home': None, 'away': None}, 'penalties': {'home': 0, 'away': 0}, 'possessiontime': {'home': 37, 'away': 63}, 'redcards': {'home': 0, 'away': 0}, 'saves': {'home': None, 'away': None}, 'shotsBlocked': {'home': None, 'away': None}, 'shotsInsidebox': {'home': None, 'away': None}, 'shotsOffgoal': {'home': 1, 'away': 6}, 'shotsOngoal': {'home': 0, 'away': 7}, 'shotsOutsidebox': {'home': None, 'away': None}, 'substitutions': {'home': 0, 'away': 1}, 'tackles': {'home': None, 'away': None}, 'throwIn': {'home': None, 'away': None}, 'yellowcards': {'home': 0, 'a

In [53]:
for game in dic_response['data']:
        date = game['date']

        awayTeam = game['awayTeam']['name']
        homeTeam = game['homeTeam']['name']
        leagueName = game['league']['name']
        countryName = game['league']['countryName']
        league = f'{countryName} - {leagueName}'
        minute = game['currentTime']['minute']
        second = game['currentTime']['second']
        awayTeamScore = game['scores']['awayTeamScore']
        homeTeamScore = game['scores']['homeTeamScore']

        try:
            injuryTime = int(game['currentTime']['injuryTime'])
            extraMinute = int(game['currentTime']['extraMinute'])
        except:
            injuryTime = 0
            extraMinute = 0

        
        if minute == None:
            minute = 0
        if second == None:
            second = 0
        
        minute = f'{minute}.{second}'
        minute = float(minute)

        status = game['status']

        if game['stats'] == None:
            continue

        iD = game['stats']['fixtureId']

        corners_home = game['stats']['corners']['home']
        corners_away = game['stats']['corners']['away']

        fouls_home = game['stats']['fouls']['home']
        fouls_away = game['stats']['fouls']['away']

        goals_home = game['stats']['goals']['home']
        goals_away = game['stats']['goals']['away']

        offsides_home = game['stats']['offsides']['home']
        offsides_away = game['stats']['offsides']['away']

        possessiontime_home = game['stats']['possessiontime']['home']
        possessiontime_away = game['stats']['possessiontime']['away']

        redcards_home = game['stats']['redcards']['home']
        redcards_away = game['stats']['redcards']['away']

        shotsOffgoal_home = game['stats']['shotsOffgoal']['home']
        shotsOffgoal_away = game['stats']['shotsOffgoal']['away']

        shotsOngoal_home = game['stats']['shotsOngoal']['home']
        shotsOngoal_away = game['stats']['shotsOngoal']['away']

        shotsBlocked_home = game['stats']['shotsBlocked']['home']
        shotsBlocked_away = game['stats']['shotsBlocked']['away']

        tackles_home = game['stats']['tackles']['home']
        tackles_away = game['stats']['tackles']['away']

        yellowcards_home = game['stats']['yellowcards']['home']
        yellowcards_away = game['stats']['yellowcards']['away']

        novo_dado = {
            'goal_home': goals_home,
            'goal_away': goals_away,
            'minute': minute,
            'league': league,
            'corners_home': corners_home,
            'corners_away': corners_away,
            'possessiontime_home': possessiontime_home,
            'possessiontime_away': possessiontime_away,
            'redcards_home': redcards_home,
            'redcards_away': redcards_away,
            'shotsOngoal_home': shotsOngoal_home,
            'shotsOngoal_away': shotsOngoal_away,
            'shotsOffgoal_home': shotsOffgoal_home,
            'shotsOffgoal_away': shotsOffgoal_away,
            'yellowcards_home': yellowcards_home,
            'yellowcards_away': yellowcards_away,
            'fouls_home': fouls_home,
            'fouls_away': fouls_away,
            'offsides_home': offsides_home,
            'offsides_away': offsides_away,
            'tackles_home': tackles_home,
            'tackles_away': tackles_away,
            'blockedShots_home': shotsBlocked_home,
            'blockedShots_away': shotsBlocked_away,
            

        }
    
        print(novo_dado)

        Xht = pd.DataFrame(novo_dado, index=[0])
        Xht.fillna(0, inplace=True)
        
        Xht['shots_home'] = Xht['shotsOngoal_home'] + Xht['shotsOffgoal_home']
        Xht['shots_away'] = Xht['shotsOngoal_away'] + Xht['shotsOffgoal_away']

{'goal_home': 0, 'goal_away': 1, 'minute': 50.43, 'league': 'Austria - Austrian Cup', 'corners_home': 1, 'corners_away': 6, 'possessiontime_home': 37, 'possessiontime_away': 63, 'redcards_home': 0, 'redcards_away': 0, 'shotsOngoal_home': 0, 'shotsOngoal_away': 7, 'shotsOffgoal_home': 1, 'shotsOffgoal_away': 6, 'yellowcards_home': 0, 'yellowcards_away': 0, 'fouls_home': None, 'fouls_away': None, 'offsides_home': None, 'offsides_away': None, 'tackles_home': None, 'tackles_away': None, 'blockedShots_home': None, 'blockedShots_away': None}
{'goal_home': 0, 'goal_away': 0, 'minute': 9.4, 'league': 'World - Club Friendlies 1', 'corners_home': 0, 'corners_away': 1, 'possessiontime_home': 58, 'possessiontime_away': 42, 'redcards_home': 0, 'redcards_away': 0, 'shotsOngoal_home': 0, 'shotsOngoal_away': 0, 'shotsOffgoal_home': 0, 'shotsOffgoal_away': 1, 'yellowcards_home': 0, 'yellowcards_away': 0, 'fouls_home': None, 'fouls_away': None, 'offsides_home': None, 'offsides_away': None, 'tackles_home

In [50]:
novo_dado

{'goal_home': 0,
 'goal_away': 0,
 'minute': 42.7,
 'league': 'Israel - Toto Cup Ligat Al',
 'corners_home': 2,
 'corners_away': 2,
 'possessiontime_home': 48,
 'possessiontime_away': 52,
 'redcards_home': 0,
 'redcards_away': 0,
 'shotsOngoal_home': 4,
 'shotsOngoal_away': 3,
 'shotsOffgoal_home': 3,
 'shotsOffgoal_away': 4,
 'yellowcards_home': 0,
 'yellowcards_away': 0,
 'fouls_home': None,
 'fouls_away': None,
 'offsides_home': None,
 'offsides_away': None,
 'tackles_home': None,
 'tackles_away': None,
 'blockedShots_home': None,
 'blockedShots_away': None}